# Music Generation with xLSTM + REMIGEN

This notebook generates music using our trained xLSTM model and decodes
REMIGEN tokens to MIDI.

Once the `xlstm` conda environment is set up, make sure to install the `midiProcessor`.

```python
cd ./repos/MidiProcessor
pip install .
pip install miditoolkit==0.1.16 numpy scipy pretty_midi mido tqdm
```

In [29]:
import sys
sys.path.append("/scratch1/e20-fyp-xlstm-music-generation/e20fyptemp1/fyp-musicgen/repos/helibrunna")

from source.languagemodel import LanguageModel
import midiprocessor as mp
import os
from pathlib import Path

print("✓ Imports successful")

✓ Imports successful


## Load xLSTM Model

In [59]:
model_path = "/scratch1/e20-fyp-xlstm-music-generation/e20fyptemp1/fyp-musicgen/repos/helibrunna/output/lmd_remigen_xlstm/run_20260106-1858"

model = LanguageModel(
    model_path,
    config_overrides={"context_length": 8192},  # Use full context
    device="cuda"  # or "cpu" if no GPU
)

model.summary()

   ▄█    █▄       ▄████████  ▄█        ▄█  ▀█████████▄     ▄████████ ███    █▄  ███▄▄▄▄   ███▄▄▄▄      ▄████████ 
  ███    ███     ███    ███ ███       ███    ███    ███   ███    ███ ███    ███ ███▀▀▀██▄ ███▀▀▀██▄   ███    ███ 
  ███    ███     ███    █▀  ███       ███▌   ███    ███   ███    ███ ███    ███ ███   ███ ███   ███   ███    ███ 
 ▄███▄▄▄▄███▄▄  ▄███▄▄▄     ███       ███▌  ▄███▄▄▄██▀   ▄███▄▄▄▄██▀ ███    ███ ███   ███ ███   ███   ███    ███ 
▀▀███▀▀▀▀███▀  ▀▀███▀▀▀     ███       ███▌ ▀▀███▀▀▀██▄  ▀▀███▀▀▀▀▀   ███    ███ ███   ███ ███   ███ ▀███████████ 
  ███    ███     ███    █▄  ███       ███    ███    ██▄ ▀███████████ ███    ███ ███   ███ ███   ███   ███    ███ 
  ███    ███     ███    ███ ███▌    ▄ ███    ███    ███   ███    ███ ███    ███ ███   ███ ███   ███   ███    ███ 
  ███    █▀      ██████████ █████▄▄██ █▀   ▄█████████▀    ███    ███ ████████▀   ▀█   █▀   ▀█   █▀    ███    █▀  
                            ▀                             ███    ███                    

/home/e20037/miniconda/envs/xlstm/lib/python3.11/site-packages/xlstm/blocks/slstm/cell.py:543: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @conditional_decorator(
/home/e20037/miniconda/envs/xlstm/lib/python3.11/site-packages/xlstm/blocks/slstm/cell.py:568: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @conditional_decorator(
/home/e20037/miniconda/envs/xlstm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:2165: FutureWarning: Calling PreTrainedTokenizerFast.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected

xLSTMLMModel(
  (xlstm_block_stack): xLSTMBlockStack(
    (blocks): ModuleList(
      (0-2): 3 x mLSTMBlock(
        (xlstm_norm): LayerNorm()
        (xlstm): mLSTMLayer(
          (proj_up): Linear(in_features=256, out_features=1024, bias=False)
          (q_proj): LinearHeadwiseExpand(in_features=512, num_heads=128, expand_factor_up=1, bias=False, trainable_weight=True, trainable_bias=True, )
          (k_proj): LinearHeadwiseExpand(in_features=512, num_heads=128, expand_factor_up=1, bias=False, trainable_weight=True, trainable_bias=True, )
          (v_proj): LinearHeadwiseExpand(in_features=512, num_heads=128, expand_factor_up=1, bias=False, trainable_weight=True, trainable_bias=True, )
          (conv1d): CausalConv1d(
            (conv): Conv1d(512, 512, kernel_size=(4,), stride=(1,), padding=(3,), groups=512)
          )
          (conv_act_fn): SiLU()
          (mlstm_cell): mLSTMCell(
            (igate): Linear(in_features=1536, out_features=4, bias=True)
            (fgate)

/home/e20037/miniconda/envs/xlstm/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Generation Parameters

Settings for controlling the music generation.

In [60]:
# start_prompt = "s-9 o-0 t-38 i-35 p-62 d-2 v-22 o-6 t-38 i-35 p-62 d-2 v-17"

# Force specific instruments in the prompt
start_prompt = "s-9 o-0 t-38 i-0 p-60 d-4 v-20 i-33 p-48 d-4 v-20 i-128 p-170 d-2 v-20"
#                        ^^^ Piano  ^^^ Bass       ^^^ Drums

In [ ]:
# Temperature: controls randomness (0.5-1.5 recommended)
# Lower = more predictable, Higher = more creative
temperature = 0.7

# Maximum length in tokens
max_length = 8000  # Full context length

# Number of songs to generate
num_songs = 1

# Starting prompt (standard REMIGEN opening)
# This tells the model: "Start a new song in 9/8 time at tempo 38"
# start_prompt = "s-9 o-0 t-38"

# Output directory
output_dir = Path("./generated_music")
output_dir.mkdir(exist_ok=True)

print(f"Temperature: {temperature}")
print(f"Max length: {max_length} tokens")
print(f"Starting prompt: {start_prompt}")
print(f"Output directory: {output_dir}")

Temperature: 0.7
Max length: 8000 tokens
Starting prompt: s-9 o-0 t-38 i-0 p-60 d-4 v-20 i-33 p-48 d-4 v-20 i-128 p-170 d-2 v-20
Output directory: generated_music


## Token Generation Function

Generates REMIGEN tokens from the model.

In [63]:
def generate_remigen_tokens(
    model,
    prompt="s-9 o-0 t-38",
    temperature=0.8,
    max_length=2048,
    stop_at_bars=None,
    verbose=True
):
    """
    Generate REMIGEN tokens from the xLSTM model.
    
    Args:
        model: The LanguageModel instance
        prompt: Starting prompt (should start with s-X o-0 t-X)
        temperature: Sampling temperature
        max_length: Maximum number of tokens to generate
        stop_at_bars: If set, stop after generating N bars (b-1 tokens)
        verbose: Print progress information
    
    Returns:
        (tokens_string, info_dict)
    """
    
    if verbose:
        print(f"🎵 Starting generation...")
        print(f"   Prompt: {prompt}")
        print(f"   Max tokens: {max_length:,}")
        print(f"   Temperature: {temperature}")
        if stop_at_bars:
            print(f"   Target bars: {stop_at_bars}")
        print()
    
    # Generate with the model
    import time
    start_time = time.time()
    
    if verbose:
        print("⏳ Generating tokens...", end="", flush=True)
    
    output_dict = model.generate(
        prompt=prompt,
        temperature=temperature,
        max_length=max_length,
        end_tokens=[],
        forbidden_tokens=["[PAD]", "[EOS]"],
        return_structured_output=True
    )
    
    elapsed = time.time() - start_time
    
    if verbose:
        print(f" Done! ({elapsed:.1f}s)")
    
    # Extract generated tokens
    tokens = output_dict["output"]
    token_list = tokens.split()
    
    if verbose:
        print(f"📊 Generated {len(token_list):,} raw tokens")
    
    # FILTER OUT INVALID TOKENS
    valid_tokens = []
    invalid_count = 0
    
    for token in token_list:
        # Only keep tokens with format: prefix-value
        if '-' in token and not token.startswith('['):
            valid_tokens.append(token)
        else:
            invalid_count += 1
            if verbose and invalid_count <= 5:  # Show first 5
                print(f"⚠️  Filtered invalid token: {token}")
    
    if verbose and invalid_count > 5:
        print(f"⚠️  Filtered {invalid_count - 5} more invalid tokens...")
    
    # Count bars in valid tokens
    bar_count = sum(1 for t in valid_tokens if t == "b-1")
    
    if verbose:
        print(f"✓ Valid tokens: {len(valid_tokens):,}")
        print(f"✓ Bars generated: {bar_count}")
    
    # Rejoin
    tokens = " ".join(valid_tokens)
    
    # Optional: truncate at bar limit
    if stop_at_bars is not None:
        if verbose:
            print(f"✂️  Truncating to {stop_at_bars} bars...")
        
        truncated = []
        bars_seen = 0
        
        for token in valid_tokens:
            truncated.append(token)
            if token == "b-1":
                bars_seen += 1
                if bars_seen >= stop_at_bars:
                    break
        
        tokens = " ".join(truncated)
        
        if verbose:
            print(f"✓ Truncated to {len(truncated):,} tokens ({bars_seen} bars)")
    
    # Add extra info to output dict
    output_dict.update({
        "valid_tokens": len(valid_tokens),
        "invalid_tokens": invalid_count,
        "bars": bar_count,
        "elapsed_time": elapsed
    })
    
    if verbose:
        print(f"⚡ Speed: {len(valid_tokens)/elapsed:.1f} tokens/sec")
        print()
    
    return tokens, output_dict

In [64]:
# Test generation
print("Testing generation...")
test_tokens, test_info = generate_remigen_tokens(
    model,
    prompt=start_prompt,
    temperature=temperature,
    max_length=200,  # Short test
    verbose=True
)

print(f"Generated {len(test_tokens.split())} tokens")
print(f"Speed: {test_info['tokens_per_second']:.2f} tokens/sec")
print(f"\nFirst 100 chars: {test_tokens[:100]}")

Testing generation...
🎵 Starting generation...
   Prompt: s-9 o-0 t-38 i-0 p-60 d-4 v-20 i-33 p-48 d-4 v-20 i-128 p-170 d-2 v-20
   Max tokens: 200
   Temperature: 0.7

⏳ Generating tokens... Done! (4.1s)
📊 Generated 200 raw tokens
✓ Valid tokens: 200
✓ Bars generated: 1
⚡ Speed: 49.3 tokens/sec

Generated 200 tokens
Speed: 45.67 tokens/sec

First 100 chars: s-9 o-0 t-38 i-0 p-60 d-4 v-20 i-33 p-48 d-4 v-20 i-128 p-170 d-2 v-20 p-164 d-4 v-20 o-6 t-38 i-0 p


## Decode Function

REMIGEN → MIDI Decoder

Converts generated REMIGEN tokens to MIDI files.

In [65]:
def decode_remigen_to_midi(token_string, output_path):
    """
    Decode REMIGEN tokens to MIDI file.
    
    Args:
        token_string: Space-separated REMIGEN tokens
        output_path: Path to save .mid file
    
    Returns:
        True if successful, False otherwise
    """
    try:
        # Split tokens
        tokens = token_string.strip().split()
        
        # Initialize decoder
        decoder = mp.MidiDecoder('REMIGEN')
        
        # Decode to MIDI object
        midi_obj = decoder.decode_from_token_str_list(tokens)
        
        # Save
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        midi_obj.dump(output_path)
        
        return True
    
    except Exception as e:
        print(f"Error decoding: {e}")
        return False

In [66]:
# Test decoding
test_output_path = output_dir / "test_decode.mid"
success = decode_remigen_to_midi(test_tokens, str(test_output_path))

if success:
    print(f"✓ Test decode successful: {test_output_path}")
else:
    print("✗ Test decode failed")

✓ Test decode successful: generated_music/test_decode.mid


## Generate Multiple Songs
Generate a batch of music samples.

In [67]:
print(f"Generating {num_songs} songs...\n")

generated_songs = []

for i in range(num_songs):
    print(f"Generating song {i+1}/{num_songs}...")
    
    # Generate tokens
    tokens, info = generate_remigen_tokens(
        model,
        prompt=start_prompt,
        temperature=temperature,
        max_length=max_length,
        stop_at_bars=32  # Generate 32 bars per song
    )
    
    # Save info
    song_data = {
        "id": i,
        "tokens": tokens,
        "num_tokens": len(tokens.split()),
        "generation_time": info["elapsed_time"],
        "tokens_per_sec": info["tokens_per_second"]
    }
    generated_songs.append(song_data)
    
    print(f"  Generated {song_data['num_tokens']} tokens in {song_data['generation_time']:.2f}s")
    print(f"  Speed: {song_data['tokens_per_sec']:.2f} tokens/sec\n")

print(f"✓ Generated {len(generated_songs)} songs")

Generating 2 songs...

Generating song 1/2...
🎵 Starting generation...
   Prompt: s-9 o-0 t-38 i-0 p-60 d-4 v-20 i-33 p-48 d-4 v-20 i-128 p-170 d-2 v-20
   Max tokens: 8,000
   Temperature: 0.7
   Target bars: 32

⏳ Generating tokens...

KeyboardInterrupt: 

### Decode All Generated Songs to MIDI

Convert all generated token sequences to MIDI files.

In [55]:
print("Decoding to MIDI files...\n")

successful = 0
failed = 0

for song_data in generated_songs:
    song_id = song_data["id"]
    tokens = song_data["tokens"]
    
    # Output path
    midi_path = output_dir / f"generated_song_{song_id:03d}.mid"
    
    # Decode
    success = decode_remigen_to_midi(tokens, str(midi_path))
    
    if success:
        successful += 1
        print(f"✓ Song {song_id}: {midi_path.name}")
    else:
        failed += 1
        print(f"✗ Song {song_id}: Failed to decode")

print(f"\n✓ Successfully decoded: {successful}/{len(generated_songs)}")
if failed > 0:
    print(f"✗ Failed: {failed}/{len(generated_songs)}")


Decoding to MIDI files...

✓ Song 0: generated_song_000.mid
✓ Song 1: generated_song_001.mid

✓ Successfully decoded: 2/2


## Analyze Generated MIDI Files

In [56]:
import pretty_midi

print("Analyzing generated music...\n")

for song_data in generated_songs[:3]:  # Analyze first 3 songs
    song_id = song_data["id"]
    midi_path = output_dir / f"generated_song_{song_id:03d}.mid"
    
    if not midi_path.exists():
        continue
    
    # Load MIDI
    midi = pretty_midi.PrettyMIDI(str(midi_path))
    
    print(f"Song {song_id}:")
    print(f"  Duration: {midi.get_end_time():.2f}s")
    print(f"  Instruments: {len(midi.instruments)}")
    print(f"  Total notes: {sum(len(inst.notes) for inst in midi.instruments)}")
    
    # Show instruments
    for inst in midi.instruments:
        inst_type = "Drums" if inst.is_drum else f"Program {inst.program}"
        print(f"    {inst.name}: {len(inst.notes)} notes ({inst_type})")
    print()

Analyzing generated music...

Song 0:
  Duration: 24.64s
  Instruments: 4
  Total notes: 412
    0: 42 notes (Program 0)
    2: 103 notes (Program 2)
    33: 188 notes (Program 33)
    128: 79 notes (Drums)

Song 1:
  Duration: 19.84s
  Instruments: 6
  Total notes: 509
    0: 270 notes (Program 0)
    6: 3 notes (Program 6)
    25: 59 notes (Program 25)
    33: 24 notes (Program 33)
    40: 27 notes (Program 40)
    128: 126 notes (Drums)



## Save Generated Tokens

Save raw token sequences for analysis.

In [57]:


tokens_dir = output_dir / "tokens"
tokens_dir.mkdir(exist_ok=True)

for song_data in generated_songs:
    song_id = song_data["id"]
    tokens = song_data["tokens"]
    
    token_path = tokens_dir / f"generated_song_{song_id:03d}.txt"
    
    with open(token_path, 'w') as f:
        f.write(tokens)

print(f"✓ Saved token files to {tokens_dir}")

✓ Saved token files to generated_music/tokens


## Generation Summary

Summary of the music generation session.

In [58]:
print("=" * 60)
print("MUSIC GENERATION SUMMARY")
print("=" * 60)
print(f"Model: {model_path}")
print(f"Songs generated: {len(generated_songs)}")
print(f"Temperature: {temperature}")
print(f"Max tokens: {max_length}")
print(f"Output directory: {output_dir}")
print(f"\nGenerated files:")
print(f"  MIDI files: {output_dir}/*.mid")
print(f"  Token files: {tokens_dir}/*.txt")
print("=" * 60)

MUSIC GENERATION SUMMARY
Model: /scratch1/e20-fyp-xlstm-music-generation/e20fyptemp1/fyp-musicgen/repos/helibrunna/output/lmd_remigen_xlstm/run_20260106-1858
Songs generated: 2
Temperature: 0.8
Max tokens: 2048
Output directory: generated_music

Generated files:
  MIDI files: generated_music/*.mid
  Token files: generated_music/tokens/*.txt
